In [1]:
#Before using spark, need to install and set up spark, Hadoop and Jdk locally on your laptop


from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("etl") \
    .config("spark.master", "local") \
    .config("spark.jars", "F:/UniPrj/Big Data Project/postgresql-42.2.16.jar") \
    .getOrCreate()

#Load Amazon Data into Spark DataFrame
#1 load product segment - Music Segment --> download dataset tsv file
file_path = "F:/UniPrj/Big Data Project/Datasets/amazon_reviews_us_Music_v1_00.tsv"
df = spark.read.option("encoding", "UTF-8").csv(file_path, sep="\t", header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [2]:
#Create DataFrame - Perform Preliminary Cleaning

#1 Check the schema, print row & column count
df.printSchema()
print((df.count(), len(df.columns)))

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)

(4751577, 15)


In [3]:
#2 Drop the round 1 columns
columns_to_drop = ['marketplace', 'product_parent', 'vine', 'review_headline', 'review_body', 'review_date']
df_dropped = df.drop(*columns_to_drop)
df_dropped.show()

+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+-----------------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|verified_purchase|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+-----------------+
|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|Whatever's for Us...|           Music|          5|            0|          0|                Y|
|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|Same Trailer Diff...|           Music|          5|            0|          0|                Y|
|   45946560| R9PYL3OYH55QY|B001GCZXW6| Soaring (Jazz Club)|           Music|          5|            0|          1|                Y|
|   15146326|R3PWBAWUS4NT0Q|B000003EK6|     CARIBBEAN PARTY|           Music|          3|            0|          0|                Y|
|   16794688|R15LYP3O51UU9E|B00N1F0BKK|         Pain Killer|  

In [4]:
#3 Filter Step 1:
df_filtered = df_dropped.filter(df_dropped.verified_purchase == 'Y')
df_filtered.show
print((df_filtered.count(), len(df_filtered.columns)))

(1962351, 9)


In [5]:
#4 drop filtered verified_purchase column
columns_to_drop = ['verified_purchase']
df_dropped_2 = df_filtered.drop(*columns_to_drop)
df_dropped_2.show()

+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+
|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|Whatever's for Us...|           Music|          5|            0|          0|
|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|Same Trailer Diff...|           Music|          5|            0|          0|
|   45946560| R9PYL3OYH55QY|B001GCZXW6| Soaring (Jazz Club)|           Music|          5|            0|          1|
|   15146326|R3PWBAWUS4NT0Q|B000003EK6|     CARIBBEAN PARTY|           Music|          3|            0|          0|
|   16794688|R15LYP3O51UU9E|B00N1F0BKK|         Pain Killer|           Music|          5|            0|          0|
|   32203364|R1AD7L0CC3DSRI|B00V7KAO7Q|A Thoughtiverse U...|           M

In [6]:
#Create Analysis-Specific DFs/Tables
#Perform Analysis-Specific Transforms
#Segmentation Analysis DF

# 1 Create Segmentation DF by Droppig Addtional Columns
segmentation_cols_drop = ['review_id', 'product_id', 'product_title', 'star_rating', 'helpful_votes', 'total_votes']
segmentation_dropped_df = df_dropped_2.drop(*segmentation_cols_drop)
segmentation_dropped_df.show()

+-----------+----------------+
|customer_id|product_category|
+-----------+----------------+
|   10140119|           Music|
|   27664622|           Music|
|   45946560|           Music|
|   15146326|           Music|
|   16794688|           Music|
|   32203364|           Music|
|    1194276|           Music|
|   45813052|           Music|
|   12795687|           Music|
|   36673840|           Music|
|   49453576|           Music|
|    3285047|           Music|
|   24471201|           Music|
|   28049396|           Music|
|   41137196|           Music|
|   43305401|           Music|
|   50808489|           Music|
|   32536909|           Music|
|   17034306|           Music|
|   49997672|           Music|
+-----------+----------------+
only showing top 20 rows



In [7]:
#Note: must change Category Label name in withColumnRenamed('count(product_category)', 'name')
#2 Segmentation GroupBy
#2a GroupBy customer_id
#2b Count product_category and rename count columns as Segment Name
segment_df = segmentation_dropped_df.groupby("customer_id")\
.agg({'product_category':'count'}).withColumnRenamed('count(product_category)', 'music')
#2c Check results
segment_df.show()

+-----------+-----+
|customer_id|music|
+-----------+-----+
|     651796|    2|
|   44979559|    1|
|   45632184|    1|
|   15232809|    1|
|   17067926|    1|
|   26163929|    1|
|   20587872|    2|
|   14230926|    2|
|   52081222|    2|
|   13382150|    1|
|   13517415|    1|
|   19324145|   26|
|   46311034|    2|
|    5573238|    1|
|   53095124|    9|
|     172523|    1|
|     283137|    1|
|    1356209|    1|
|   28074100|    2|
|   34722529|    1|
+-----------+-----+
only showing top 20 rows



In [8]:
#3 Check segment_df Schema and Row Count
segment_df.printSchema()
print(segment_df.count())

root
 |-- customer_id: integer (nullable = true)
 |-- music: long (nullable = false)

886919


In [9]:
#Note: Column Name in df.sort('name' ...) must align with Column name from step 2
#4 Filter for Top n Results
#4a Declare number of rows to filter by (100,000)
row_count = 100000
#4a Sort by Segment Desc and limit to row_count
filtered_segment_df = segment_df.sort('music', ascending=False).limit(row_count)
#4b Check Results
filtered_segment_df.show()
print(filtered_segment_df.count())

+-----------+-----+
|customer_id|music|
+-----------+-----+
|   29791894| 1089|
|   51184997|  984|
|   47423754|  976|
|   38192329|  881|
|   52562189|  850|
|   27364030|  821|
|   49939297|  775|
|   52469795|  774|
|   52467002|  742|
|   47883385|  716|
|   51228286|  679|
|   49877557|  595|
|   18116317|  549|
|   50910905|  480|
|   50135456|  469|
|   50345651|  462|
|   53075795|  440|
|   15536614|  414|
|   45772507|  413|
|   44861557|  409|
+-----------+-----+
only showing top 20 rows

100000


In [10]:
#Segmentation ETL Complete - Add Database Export Code

# Configure settings for RDS
mode = "append"
jdbc_url = 'jdbc:postgresql://localhost:5432/AmazonProducts'
config = {"user":'postgres', 
          "password": 'root', 
          "driver":"org.postgresql.Driver"}

In [11]:
#Note: table name in table='name_segment' must align with table name in Postgres

# Write segment table to Postgres/RDS
# xx mins
filtered_segment_df.write.jdbc(url=jdbc_url, table='music_segment', mode=mode, properties=config)

In [12]:
# Run Queries in Postgres to Confirm Segment ETL
# Check Row Count of Segment Table - SELECT COUNT(*) FROM music_segment;
# Check 10 Rows of Segment Table - SELECT * FROM music_segment LIMIT(10);
# Upon Confirmation of Above Checks Run This Cell
print('Segment ETL Successful')
     

Segment ETL Successful


In [13]:
# Apriori Analysis DF
# Enter Apriori Transfomations Below - Use df_dropped_2 as Start Point
# 1 Create Apriori DF by First Dropping Addtional Columns
apriori_cols_drop = ['product_category', 'product_title', 'star_rating', 'helpful_votes', 'total_votes']
apriori_dropped_df = df_dropped_2.drop(*apriori_cols_drop)
apriori_dropped_df.show()

+-----------+--------------+----------+
|customer_id|     review_id|product_id|
+-----------+--------------+----------+
|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|
|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|
|   45946560| R9PYL3OYH55QY|B001GCZXW6|
|   15146326|R3PWBAWUS4NT0Q|B000003EK6|
|   16794688|R15LYP3O51UU9E|B00N1F0BKK|
|   32203364|R1AD7L0CC3DSRI|B00V7KAO7Q|
|    1194276|R32FE8Y45QV434|B000094Q4P|
|   45813052|R3NM4MZ4XWL43Q|B00JMK0P1I|
|   12795687|R3H4FXX6Q7I37D|B008OW1S3O|
|   36673840|R30L5PET7LFFDC|B00VI2L3L4|
|   49453576|  REFRE1LEKLAF|B0000041EV|
|    3285047|R3JTJ5EQN74E9H|B00005YW4H|
|   24471201|R1W2F091LCOAW5|B00Q9KEZV0|
|   28049396| RYUMFQRRB1FNM|B00GFXRKHW|
|   41137196| RHCS6VVXWV3Q3|B004L3AQ10|
|   43305401|R35UL1YZ2142SL|B0000025PM|
|   50808489|  RU1O6QD1LEBB|B00701QUN8|
|   32536909|R16O7SHYHLEYA1|B00LGA16J2|
|   17034306|R25Z4ADMYBHR66|B00AIUSGJE|
|   49997672| RS04EK5PRDM1H|B003X43FT8|
+-----------+--------------+----------+
only showing top 20 rows



In [14]:
#Prelim Apriori ETL Complete - Add Database Export Code
# Configure settings for RDS
mode = "append"
jdbc_url = 'jdbc:postgresql://localhost:5432/AmazonProducts'
config = {"user":'postgres', 
          "password": 'root', 
          "driver":"org.postgresql.Driver"}

In [15]:
#Note: table name in table='segment_apriori' must align with table name in Postgres
# Write segment_apriori table to RDS
# 5 mins
apriori_dropped_df.write.jdbc(url=jdbc_url, table='music_apriori', mode=mode, properties=config)     


In [16]:
# Run Queries in Postgres to Confirm Apriori ETL
# Check Row Count of Apriori Table - SELECT COUNT(*) FROM music_apriori;
# Check 10 Rows of Apriori Table - SELECT * FROM music_apriori LIMIT(10);
# Upon Confirmation of Above Checks Run This Cell
print('Apriori ETL Successful')

Apriori ETL Successful
